In [ ]:
import json
import os

In [ ]:
input_filename = 'jsonexample.txt' # Replace with the actual name of your input JSON file

# Define the target model names to filter by
target_models = ["CARDAMOM", "CMS-Flux", "ECCO", "ISSM", "MOMO-Chem", "RAPID"]

# Dictionary to hold filtered publications for each model
# Initialize with empty lists for each target model
filtered_publications = {model: [] for model in target_models}

try:
    # Read the JSON data from the input file
    with open(input_filename, 'r') as f:
        data = json.load(f)

    # Verify that the loaded data is a list (JSON array)
    if not isinstance(data, list):
        print(f"Error: Input file '{input_filename}' does not contain a JSON array at the top level.")
        print("Please ensure the file is formatted as a list of JSON objects, like: [{}, {}, ...]")
    else:
        print(f"Successfully loaded {len(data)} potential publication objects from '{input_filename}'.")

        # Iterate through each publication object in the list
        for publication in data:
            # Get the value of the 'models' field, default to an empty list if not found
            # Ensure the retrieved value is treated as a list
            models_list = publication.get('models', [])
            if not isinstance(models_list, list):
                models_list = [] # Treat non-list 'models' field as empty

            # Check if any of the target models are present in this publication's models list
            for model in models_list:
                if model in target_models:
                    # Add the publication to the list for the matching model
                    # A single publication can appear in multiple model files if it lists multiple target models
                    filtered_publications[model].append(publication)


        # Create the output directory if it doesn't exist
        output_dir = "./filtered_models"
        os.makedirs(output_dir, exist_ok=True)

        # Save the filtered publications to separate JSON files for each model
        print(f"\nSaving filtered publications to the '{output_dir}' directory...")
        for model_name, publications_list in filtered_publications.items():
            if publications_list:
                output_filename = os.path.join(output_dir, f"{model_name}.json")
                try:
                    with open(output_filename, 'w') as outfile:
                        # Use indent=4 for pretty printing the JSON
                        json.dump(publications_list, outfile, indent=4)
                    print(f"Saved {len(publications_list)} publications for model '{model_name}' to '{output_filename}'.")
                except IOError as e:
                    print(f"Error writing file '{output_filename}': {e}")
            else:
                print(f"No publications found for model '{model_name}'. Skipping file creation.")

except FileNotFoundError:
    print(f"Error: Input file '{input_filename}' not found.")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from '{input_filename}'. Please ensure it is a valid JSON file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")